<a href="https://colab.research.google.com/github/LanaRozman/Homework/blob/main/memes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание с мемами

Импорт библиотек:

In [69]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
!pip install texthero
 
import texthero as hero

Мимикрирую под человека:

In [ ]:
!pip install fake_useragent

In [ ]:
from fake_useragent import UserAgent
print(UserAgent().chrome)

In [143]:
page_link = 'https://knowyourmeme.com/memes/all/page/1'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

Список, в котором будут названия мемов:

In [144]:
titles = []

Циклом прохожу по всем страницам, на всякий сучай используя time, из таблички с мемами беру то, что лежит в h2 (там ссылки и заголовки), а потом оттуда достаю текст ссылки и добавляю в список

In [145]:
for i in range(1,15):
    page_response = requests.get(f'https://knowyourmeme.com/memes/all/page/{i}/',headers={'User-Agent': UserAgent().chrome})
    time.sleep(0.3)
    soup = BeautifulSoup(page_response.text, 'html.parser')
    table = soup.find('table',class_="entry_list")
    element = table.find_all('h2')
    for elem in element:
      title = elem.find('a').get_text(strip=True)
      titles.append(title)

Положила все в датафрейм и обнаружила какой-то непонятный заголовок, которого в html страницы я не нашла, поэтому я его удалила:

In [155]:
dframe = pd.DataFrame(titles) 
dframe.columns = ['Memes']
dframe.drop(labels = [0], axis = 0, inplace = True)
dframe

,Memes
1,My Brother In Christ
2,Sibling Memes
3,"""Thinking With My Dick"" / Mardi Gras Guy"
4,Vape Girl
5,Unalive
...,...
219,Bear Chasing Person Down Hall
220,Enough Champagne To Fill The Nile
221,Donald Trump Eating / Flushing Classified Docu...
222,Gunt


Я решила попытаться проанализировать мемы, для этого я их очистила от всех служебных символов и стоп-слов(которые уже есть в библиотеке)

In [156]:
dframe['Memes Cleaned'] = hero.clean(dframe['Memes'])
dframe

,Memes,Memes Cleaned
1,My Brother In Christ,brother christ
2,Sibling Memes,sibling memes
3,"""Thinking With My Dick"" / Mardi Gras Guy",thinking dick mardi gras guy
4,Vape Girl,vape girl
5,Unalive,unalive
...,...,...
219,Bear Chasing Person Down Hall,bear chasing person hall
220,Enough Champagne To Fill The Nile,enough champagne fill nile
221,Donald Trump Eating / Flushing Classified Docu...,donald trump eating flushing classified docume...
222,Gunt,gunt


Визуализируем топ частотных слов:

In [165]:
topwords = hero.visualization.top_words(dframe['Memes Cleaned']).head(10)

import plotly.express as px

fig = px.bar(topwords)
fig.show()
topwords.head()

vs         6
russian    5
super      4
life       3
day        3
Name: Memes Cleaned, dtype: int64

Тут мы можем сделать такие выводы: 


1.   Мемы часто строятся на оппозиции (vs)
2.   Политика и повседневная жизнь - наиболее популярные источники мемов (day, life, happy, russian, ukranian).


Изначально в визуализации в топе было слово "x",чаще всего оно не несет в мемах смысловой нагрузки , поэтому я добавила его в список стоп-слов:

In [166]:
from texthero import stopwords

default_stopwords = stopwords.DEFAULT
custom_stopwords = default_stopwords.union(set(["x"]))
dframe['Memes Cleaned'] = hero.remove_stopwords(dframe['Memes Cleaned'], custom_stopwords)